## Install pyXNAT

pyXNAT provides a programmatic interface with XNAT.

In [ ]:
! pip install pyxnat

## Create directory to store DICOM data

In [ ]:
import os

#location of input DICOM folders
workDir = r'/content' #for Colab
inputDicomPath = os.path.join(workDir,'input')
os.makedirs(inputDicomPath, exist_ok=True)

## Download data from XNAT source


In [ ]:
from pyxnat import Interface
from glob import glob
import urllib3
import shutil
urllib3.disable_warnings()

from pyxnat import Interface

def getXNATData(xhost,user,scandict,downloadDir):
  xnat = Interface(xhost, user, verify=False)
  os.makedirs(downloadDir, exist_ok=True)
  expdirlist = []
  for scan_entry in scandict:
    proj = scan_entry['proj']
    subj = scan_entry['subj']
    exp = scan_entry['exp']
    scan_list = scan_entry['scan_list']
    expdir = os.path.join(downloadDir,exp)
    expdirlist.append(expdir)
    os.makedirs(expdir, exist_ok = True)
    xexp = xnat.select.project(proj).subject(subj).experiment(exp)
    for scan in scan_list:
      try:
        xnat.select.project(proj).subject(subj).experiment(exp).scan(scan).resource('DICOM').get(downloadDir,extract=True)
      except:
        xnat.select.project(proj).subject(subj).experiment(exp).scan(scan).resource('secondary').get(downloadDir,extract=True)
    for dcmfolder in ['DICOM','secondary']:
      dcmlist = glob(os.path.join(downloadDir,dcmfolder,'*.dcm'))
      print(dcmlist)
      for dcm in dcmlist:
        shutil.move(dcm, expdir)
  for dcmfolder in ['DICOM','secondary']:
    if os.path.exists(os.path.join(downloadDir,dcmfolder)):
      os.rmdir(os.path.join(downloadDir,dcmfolder))
    if os.path.exists(os.path.join(downloadDir,dcmfolder + '.zip')):
      os.remove(os.path.join(downloadDir,dcmfolder + '.zip'))
  xnat.disconnect()
  return expdirlist

In [ ]:
xhost = 'https://xnat.yoursite.org'
user = 'usr'
scandict = [{'proj':proj,'subj':subj,'exp':exp, 'scan_list':['1']}]

folderList = getXNATData(xhost,user,scandict,inputDicomPath)